In [4]:
import MalthusModel

model = MalthusModel.Malthus()

print(model.Lt1(560000000))

(1687882613492.6968, 3346.640106136303, 1874678459436.3298, -187355845943.633)


In [ ]:
import MalthusModel
import sympy as sym

model = MalthusModel.Malthus()

def find_steady_state_labor(model):
    return optimize.fsolve(model.L, 0)  # Finding the root of the function L(t)


# Example usage
malthus_model = Malthus()
steady_state_labor = find_steady_state_labor(malthus_model)
print("Steady-state labor (L):", steady_state_labor)